#### Check whether given CFG is LL1 or not (1 mark)

In [4]:
# Define a function to compute the First set for a given symbol
def compute_first(grammar, symbol, first_sets):
    if symbol in first_sets:
        return first_sets[symbol]

    first = set()
    if symbol not in grammar:
        first.add(symbol)
    else:
        for production in grammar[symbol]:
            if production[0] == symbol:
                continue  # Avoid left recursion

            for char in production:
                if char != symbol:
                    first.update(compute_first(grammar, char, first_sets))
                    if char not in grammar or '' not in grammar[char]:
                        break
            else:
                first.add('')

    first_sets[symbol] = first
    return first

# Define a function to compute the Follow set for a given symbol
def compute_follow(grammar, symbol, follow_sets, first_sets):
    if symbol in follow_sets:
        return follow_sets[symbol]

    follow = set()
    if symbol == grammar['S'][0][0]:  # If it's the start symbol
        follow.add('$')  # Add end-of-input marker

    for left_symbol in grammar:
        for production in grammar[left_symbol]:
            for idx, right_symbol in enumerate(production):
                if right_symbol == symbol:
                    next_symbol = None

                    # Find the next symbol in the production
                    if idx + 1 < len(production):
                        next_symbol = production[idx + 1]

                    if next_symbol:
                        # Add First(next_symbol) to Follow(symbol)
                        follow.update(compute_first(grammar, next_symbol, first_sets))

                        # If First(next_symbol) contains epsilon (''), add Follow(left_symbol) to Follow(symbol)
                        if '' in compute_first(grammar, next_symbol, first_sets):
                            follow.update(compute_follow(grammar, left_symbol, follow_sets, first_sets))
                    else:
                        # If it's the last symbol in the production, add Follow(left_symbol) to Follow(symbol)
                        follow.update(compute_follow(grammar, left_symbol, follow_sets, first_sets))

    follow_sets[symbol] = follow
    return follow

# Define a function to check if the grammar is LL(1)
def is_ll1_grammar(grammar):
    first_sets = {}
    follow_sets = {}

    for non_terminal in grammar:
        compute_first(grammar, non_terminal, first_sets)
        compute_follow(grammar, non_terminal, follow_sets, first_sets)

    parsing_table = {}

    for non_terminal in grammar:
        for production in grammar[non_terminal]:
            first = compute_first(grammar, production[0], first_sets)
            for terminal in first:
                if terminal:
                    if non_terminal not in parsing_table:
                        parsing_table[non_terminal] = {}
                    if terminal in parsing_table[non_terminal]:
                        return False  # Conflict in parsing table
                    parsing_table[non_terminal][terminal] = production

            if '' in first:
                follow = follow_sets[non_terminal]
                for terminal in follow:
                    if terminal:
                        if non_terminal not in parsing_table:
                            parsing_table[non_terminal] = {}
                        if terminal in parsing_table[non_terminal]:
                            return False  # Conflict in parsing table
                        parsing_table[non_terminal][terminal] = production

    return True  # No conflicts found in the parsing table

# Get the grammar input from the user
user_input = input("Enter the grammar (e.g., S -> aS | bS | ε, A -> cA | ε): ")

# Parse the user input to create the grammar dictionary
grammar = {}
productions = user_input.split(', ')
for production in productions:
    left, right = production.split(' -> ')
    if left not in grammar:
        grammar[left] = []
    grammar[left].append(right.split(' '))

# Check if the grammar is LL(1)
if is_ll1_grammar(grammar):
    print("The grammar is LL(1).")
else:
    print("The grammar is not LL(1) or contains conflicts.")


The grammar is LL(1).
